<a href="https://colab.research.google.com/github/missu79/sa_for_nsmc_koreauniv_project/blob/main/2019516024_jungchangwoo_nsmc_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **0. 목적**
 - 본 파일은 고려대학교 디지털금융석사과정(계약과정) 중 <br>
   '디지털금융공학을 위한 자연언어 처리기술 과정'의 기말과제를 위해 만들어진 파일입니다.
   - 과제명 : Naver sentiment movie corpus 분석

- 우종하(deepseasw)님의 소스를 전적으로 참고하였습니다.
  - 출처 : https://github.com/deepseasw/bert-naver-movie-review/blob/master/bert_naver_movie.ipynb

- 주석(텍스트 편집)과 일부 코드 편집은 정창우 본인이 직접 하였음을 밝힙니다. <br>
  한줄한줄 공부하면서 진행했습니다.
  - missu797979@gmail.com

## **1. colab 연결**
 - 구글 드라이브를 colab의 /content/gdrive/ 에 연결
 - 암호 입력(인증)
 


In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


## **2. transformer 설치**

In [2]:
!pip install transformers

## **3. 필요한 라이브러리 가져오기**
 - tf, NN, tokenizer 등 필요한 라이브러리 import

In [3]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

## **4. 데이터 불러오기**
 - 데이터 다운로드
 - 디렉토리에 어떤 파일이 있는지 확인
 - 훈련셋과 데이터셋 로드
 - 내용 확인

In [4]:
!git clone https://github.com/e9t/nsmc.git

!ls nsmc -la

DATA_PATH = 'gdrive/My Drive/nsmc/'
import sys
sys.path.append(DATA_PATH)

train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')
final_test = pd.read_csv(DATA_PATH + "ko_data.csv", sep=',', encoding='CP949')

print(train.shape)
print(test.shape)
print(final_test.shape)

train.head(10)

fatal: destination path 'nsmc' already exists and is not an empty directory.
total 38636
drwxr-xr-x 5 root root     4096 Dec 25 04:43 .
drwxr-xr-x 1 root root     4096 Dec 25 04:43 ..
drwxr-xr-x 2 root root     4096 Dec 25 04:43 code
drwxr-xr-x 8 root root     4096 Dec 25 04:43 .git
-rw-r--r-- 1 root root  4893335 Dec 25 04:43 ratings_test.txt
-rw-r--r-- 1 root root 14628807 Dec 25 04:43 ratings_train.txt
-rw-r--r-- 1 root root 19515078 Dec 25 04:43 ratings.txt
drwxr-xr-x 2 root root   458752 Dec 25 04:43 raw
-rw-r--r-- 1 root root     2596 Dec 25 04:43 README.md
-rw-r--r-- 1 root root    36746 Dec 25 04:43 synopses.json
(150000, 3)
(50000, 3)
(11187, 2)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


## **5. trian set 전처리**
 - 리뷰 문장 추출 후 형식 변환

- label 추출 후 tokenizer로 문장을 토큰으로 분리

- max_len을 128로 설정

- 토큰을 숫자로 변환 후 max_len에 맞게 자르고, 모자란 부분을 패딩 0으로 채움

- attention_mask 초기화
 - attention_mask를 패딩이 아니면 1, 패딩이면 0으로 설정
 - 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상

- 훈련셋과 검증셋으로 분리
 - attention_mask도 훈련셋과 검증셋으로 분리

- 데이터를 텐서로 변환

- batch size 지정 및 데이터 설정
 - 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
 - 학습시 배치 사이즈 만큼 데이터를 가져옴 <br>

 <font color = "RED"> **[Tip1]** <br>
 배치사이즈를 크게 하면 속도향상에는 도움이 되나, 메모리가 좋아야 가능함. <br>
 256으로 돌려보니, runtime error가 발생하였음.  안전하게 32로 세팅</font>

In [7]:
sentences = train['document']
sentences[:10]

sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

labels = train['label'].values
labels

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

[CLS] 아 더빙.. 진짜 짜증나네요 목소리 [SEP]
['[CLS]', '아', '더', '##빙', '.', '.', '진', '##짜', '짜', '##증', '##나', '##네', '##요', '목', '##소', '##리', '[SEP]']
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
tensor([   101,   9711,  11489,   9364,  41850,   9004,  32537,   9491,  35506,
         17360,  48549,    119,    119,   9477,  26444,  12692,   9665,  21789,
         11287,   9708, 119235,   9659,  22458, 119136,  

## **6. test set 전처리**
 - 리뷰 문장 추출 후 형식 변환

- label 추출 후 tokenizer로 문장을 토큰으로 분리

- max_len을 128로 설정

- 토큰을 숫자로 변환 후 max_len에 맞게 자르고, 모자란 부분을 패딩 0으로 채움

- attention_mask 초기화
 - attention_mask를 패딩이 아니면 1, 패딩이면 0으로 설정
 - 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상

- 훈련셋과 검증셋으로 분리
 - attention_mask도 훈련셋과 검증셋으로 분리

- 데이터를 텐서로 변환

- batch size 지정 및 데이터 설정
 - 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
 - 학습시 배치 사이즈 만큼 데이터를 가져옴
 

In [8]:
sentences = test['document']
sentences[:10]

final_sentences = final_test['Sentence']
final_sentences[:10]

sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
final_sentence = ["[CLS] " + str(final_sentence) + " [SEP]" for final_sentence in final_sentences]

sentences[:10]

labels = test['label'].values
labels

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
final_tokenized_texts = [tokenizer.tokenize(sent) for sent in final_sentence]

print (sentences[0])
print (tokenized_texts[0])
print (final_tokenized_texts[0])

MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
final_input_ids = [tokenizer.convert_tokens_to_ids(x) for x in final_tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
final_input_ids = pad_sequences(final_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

attention_masks = []
final_attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

for seq in final_input_ids:
    seq_mask = [float(i>0) for i in seq]
    final_attention_masks.append(seq_mask)      

print(attention_masks[0])

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)
final_test_id = torch.tensor(final_test['Id'])
final_test_inputs = torch.tensor(final_input_ids)
final_test_masks = torch.tensor(final_attention_masks)

print(final_test_id[0])
print(final_test_inputs[0])
print(final_test_masks[0])

batch_size = 32

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

final_test_data = TensorDataset(final_test_id, final_test_inputs, final_test_masks)
final_test_dataloader = DataLoader(final_test_data, batch_size=1)

[CLS] 굳 ㅋ [SEP]
['[CLS]', '굳', '[UNK]', '[SEP]']
['[CLS]', '정', '##말', '많이', '울', '##었던', '영화', '##입', '##니다', '.', '[SEP]']
[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
tensor(0)
tensor([  101,  9670, 89523, 47058,  9607, 61439, 42428, 58303, 48345,   119,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     

## **7. 모델링**
 - GPU 디바이스 이름 검사

-  디바이스 설정

-  BERT 모델 생성<br>
   <font color = "RED"> **[Tip2]** <br>
 한국어는 언어의 특성을 최적화 한 ko-BERT가 더 성능이 높게 나올 것 <br>
 다음 번에 기회가 있다면 ko-BERT를 활용하고자 함</font>

- 옵티마이저 설정 및 parameter 값 조정
 - 러닝레이트 설정하고, 0으로 나누는 것을 방지하기 위해 입실론 값 적용
 - 에폭 횟수 설정
 - 총 스텝은 배치반복 횟수 X 에폭
 - 스케쥴러 생성(러닝레이트를 조금씩 변화시킴)<br>
<font color = "RED"> **[Tip3]** <br>
 learning rate와 epoch을 조정하여 여러번 돌려볼 필요가 있음</font> 
 



In [9]:
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8 
                )

epochs = 2

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

Found GPU at: /device:GPU:0
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

## 8. 학습 및 검증
- 함수 입력
 - 정확도 계산 함수
 - 시간 계산 함수

- 랜덤 시드 고정
- gradient 초기화
- 에폭만큼 학습 수행

#ㅁ training#
  - 트레인 모델
  - 데이터로더에서 batch만큼 반복하여 가져옴
  - batch를 GPU에 넣음
  - batch에서 데이터 추출
  - forward 수행
  - loss 및 총 loss 계산
  - backward 수행
  - gradient 클리핑
  - 가중치 parameter 업데이트
  - 스케쥴러로 러닝레이트 감소
  - gradient 초기화
  - 평균 loss 구하기

#ㅁ validation#
  - 평가 모델
  - 데이터로더에서 batch만큼 반복하여 가져옴
  - batch를 GPU에 넣음
  - batch에서 데이터 추출
  - gradient 계산 안함
  - loss 구하기
  - cpu로 데이터 이동
  - 출력 로짓과 라벨을 비교하여 정확도 계산

In [10]:
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):

    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

seed_val = 27
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0

    model.train()
        
    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
          
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        loss = outputs[0]

        total_loss += loss.item()

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        scheduler.step()

        model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():     
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")   


======== Epoch 1 / 2 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:05:48.
  Batch 1,000  of  4,219.    Elapsed: 0:11:39.
  Batch 1,500  of  4,219.    Elapsed: 0:17:30.
  Batch 2,000  of  4,219.    Elapsed: 0:23:21.
  Batch 2,500  of  4,219.    Elapsed: 0:29:12.
  Batch 3,000  of  4,219.    Elapsed: 0:35:03.
  Batch 3,500  of  4,219.    Elapsed: 0:40:54.
  Batch 4,000  of  4,219.    Elapsed: 0:46:45.

  Average training loss: 0.38
  Training epcoh took: 0:49:18

Running Validation...
  Accuracy: 0.86
  Validation took: 0:01:54

======== Epoch 2 / 2 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:05:51.
  Batch 1,000  of  4,219.    Elapsed: 0:11:42.
  Batch 1,500  of  4,219.    Elapsed: 0:17:33.
  Batch 2,000  of  4,219.    Elapsed: 0:23:24.
  Batch 2,500  of  4,219.    Elapsed: 0:29:15.
  Batch 3,000  of  4,219.    Elapsed: 0:35:06.
  Batch 3,500  of  4,219.    Elapsed: 0:40:57.
  Batch 4,000  of  4,219.    Elapsed: 0:46:48.

  Average training loss: 0.28
  

## 9. 최종 제출데이타에 대한 예측 수행

- 평가 모델
  - 데이터로더에서 batch만큼 반복하여 가져옴
  - batch를 GPU에 넣음
  - batch에서 데이터 추출
  - gradient 계산 안함
  - loss 구하기
  - cpu로 데이터 이동

- 긍정(1)/부정(0)으로 분류

- 최종 제출 파일 생성

In [11]:
t0 = time.time()
batch = 1

model.eval()

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
final_result = []

for step, batch in enumerate(final_test_dataloader):
    if step % 1000 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(final_test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)
    
    b_id, b_input_ids, b_input_mask = batch
    
    with torch.no_grad():     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    b_id = b_id.cpu().numpy()

    result = np.concatenate((b_id, pred_flat), axis=None)
        
    final_result.append(result)

rdf = pd.DataFrame(final_result, columns =['Id', 'Predicted'])
rdf.to_csv(DATA_PATH + 'nsmc_final.csv', index=False)

final_result[:10]

  Batch 1,000  of  11,187.    Elapsed: 0:00:12.
  Batch 2,000  of  11,187.    Elapsed: 0:00:24.
  Batch 3,000  of  11,187.    Elapsed: 0:00:36.
  Batch 4,000  of  11,187.    Elapsed: 0:00:48.
  Batch 5,000  of  11,187.    Elapsed: 0:01:01.
  Batch 6,000  of  11,187.    Elapsed: 0:01:13.
  Batch 7,000  of  11,187.    Elapsed: 0:01:25.
  Batch 8,000  of  11,187.    Elapsed: 0:01:37.
  Batch 9,000  of  11,187.    Elapsed: 0:01:49.
  Batch 10,000  of  11,187.    Elapsed: 0:02:01.
  Batch 11,000  of  11,187.    Elapsed: 0:02:14.


[array([0, 1]),
 array([1, 0]),
 array([2, 0]),
 array([3, 1]),
 array([4, 0]),
 array([5, 1]),
 array([6, 0]),
 array([7, 1]),
 array([8, 0]),
 array([9, 1])]